In [ ]:
import tensorflow as tf
import os
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import csv
import readData
PH = 30
category = 'epoch300'
datapath = 'data/sch'

filename = os.listdir(datapath)
train_data_name = []
test_data_name = []

for fn in filename:
    if fn.find("test") != -1:
        test_data_name.append(fn)
    elif fn.find("train") != -1:
        train_data_name.append(fn)

print("Data load...")
A1cList = []
A1c_train = []
A1c_test = []

total_x_data = []
total_y_data = []        

test_x_data = []
test_y_data = []
train_x_data = []
train_y_data = []

for fn in train_data_name:
    x,y,A1c,DM,BMI,age,AD = readData.readData(datapath+"/"+str(fn), PH)
    train_x_data.append(x)
    total_x_data.append(x)
    train_y_data.append(y)
    total_y_data.append(y)
    A1cList.append(A1c)
    A1c_train.append(A1c)
    print(fn)

for fn in test_data_name:
    x,y,A1c,DM,BMI,age,AD = readData.readData(datapath+"/"+str(fn), PH)
    test_x_data.append(x)
    #total_x_data.append(x)
    test_y_data.append(y)
    #total_y_data.append(y)
    #A1cList.append(A1c)
    A1c_test.append(A1c)
    print(fn)
    
print("Data load Complete!")

In [ ]:
X = tf.placeholder(tf.float32,shape=[None,7,1])
Y = tf.placeholder(tf.float32,shape=[None,1])

'''
W = tf.Variable(tf.random_uniform([1,15], minval = -1, maxval = 1, dtype = tf.float32))
b = tf.Variable(tf.zeros([1,15],dtype = tf.float32))

W1 = tf.Variable(tf.random_uniform([15,1], minval = -1, maxval = 1, dtype = tf.float32))
b1 = tf.Variable(tf.zeros([1,1],dtype = tf.float32))'''

cell = tf.contrib.rnn.BasicLSTMCell(num_units=20,state_is_tuple=True,activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell,X,dtype=tf.float32)

Y_p = tf.contrib.layers.fully_connected(outputs[:,-1],64,activation_fn=None)
Y_p2 = tf.contrib.layers.fully_connected(Y_p,10,activation_fn=None)
O1 = tf.contrib.layers.fully_connected(Y_p2,1,activation_fn=None)

save_file = './PH30_pretrain.ckpt'
saver = tf.train.Saver()

A1c = tf.placeholder(tf.float32,shape=[None,1])
Y_A = tf.contrib.layers.fully_connected(A1c,10,activation_fn=tf.sigmoid)
O2 = tf.contrib.layers.fully_connected(Y_A,1,activation_fn=tf.sigmoid)
#Y_pr = tf.contrib.layers.fully_connected(Y_p+Y_A,1,activation_fn=None)
Y_pr = tf.contrib.layers.fully_connected(O1+O2,15,activation_fn=None)

Y_pre =  tf.contrib.layers.fully_connected(Y_pr,1,activation_fn=None)

rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(Y, Y_pre))))
loss = tf.reduce_sum(tf.square(Y_pre - Y))
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
train = optimizer.minimize(loss)

summary = tf.summary.merge_all()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print("Weight of Pre train load...")
print("--------------------------------------------------")
saver.restore(sess, save_file)
print("Weight of Pre train load ")

print("Learning start...")
print("--------------------------------------------------")
for loop in range(len(total_x_data)):
    for i in  range(300):
        sess.run(train,feed_dict = {X:train_x_data[loop],Y:train_y_data[loop],A1c:A1cList[loop]})
        #sess.run(train,feed_dict = {X:total_x_data[loop],Y:total_y_data[loop]})
    print(str(loop / len(total_x_data) * 100) + "%")

print("Learning Complete!")

In [ ]:
top = 0
patient_num = 1

print("****** 실제 혈당 / 예측 혈당 (PH " + str(PH) + "분 뒤) ******")
print("--------------------------------------------------")

pre_List = []
for loop in range(len(test_x_data)):
    for i,k in enumerate(test_x_data[patient_num]):
        ln = sess.run(Y_pre,feed_dict={X:[k],A1c:A1c_test[0]})
        if top < 5:
            print("실제 혈당 : "+str(test_y_data[patient_num][i])+" / 예측 혈당 : "+str(ln[0][0]))
            top += 1
        pre_List.append(ln[0][0])
        #print([sess.run(Y_p,feed_dict={X:[k]})[0][0],10.0])
    break;

rmse_total = 0.0
rmse_one = 0.0
total_num = 0
one_person_num = 0
for size in range(len(test_x_data)):
    one_person_num = 0
    rmse_one = 0
    for i,k in enumerate(test_x_data[size]):
        total_num += 1
        one_person_num += 1
        #print(sess.run(rmse,feed_dict={X:[k],Y:[test_y_data[size][i]],A1c:[A1c_test[size][i]],G:[glucose_test[size][i]]}))
        tmp = sess.run(rmse,feed_dict={X:[k],Y:[test_y_data[size][i]],A1c:[A1c_test[size][i]]})
        rmse_total += tmp
        rmse_one += tmp
    #print(str(size+1)+"번째 환자 rmse : "+str(rmse_one/float(one_person_num)))

rmse_total = rmse_total / float(total_num)

print("...")
print("--------------------------------------------------")
print("* 전체 평균 rmse : "+str(rmse_total))
print("* Blue : 실제 혈당 / Red : 예측 혈당")

plt.plot(pre_List,'r--')
plt.plot(test_y_data[patient_num],color="blue")
plt.show()